In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_path = '/content/drive/MyDrive/Customer_Churn/cleaned/cleaned_data.csv'

In [ ]:
def load_data(input_path):

  # Read the CSV file
  df = pd.read_csv(input_path)
  print("✅ Succesfully loaded the dataset")

  return df

In [ ]:
cleaned_data = load_data(input_path)
pd.set_option('display.max_columns', None)
cleaned_data.head()

✅ Succesfully loaded the dataset


,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag,Last_Transaction_Date,Utilization_Group
0,49,F,2,High School,Married,Less than $40K,Silver,34,4,2,0,4710.0,3137.0,1573.0,0.550,2448.0,38,0.817,0.666,Existing Customer,2023-11-05,High (30-70%)
1,30,M,2,High School,Married,$40K - $60K,Blue,30,3,1,0,6038.0,824.0,5214.0,0.520,10069.0,26,1.099,0.136,Attrited Customer,2022-08-14,Moderate (10-30%)
2,41,M,1,High School,Divorced,$40K - $60K,Blue,50,2,1,1,3924.0,3044.0,880.0,1.113,6389.0,112,0.674,0.776,Existing Customer,2023-02-12,Very High (70%+)
3,43,M,3,College,Married,$60K - $80K,Blue,40,4,3,0,9226.0,855.0,8371.0,0.157,14261.0,106,1.160,0.093,Attrited Customer,2023-01-05,Low (0-10%)
4,37,F,2,Post-Graduate,Single,Less than $40K,Blue,26,4,1,2,2734.0,1143.0,1591.0,0.736,3046.0,30,0.755,0.418,Existing Customer,2024-01-15,High (30-70%)


# Advance Feature Engineering

## Financial Health Indicators

In [ ]:
def create_financial_features(df):
  """
  Create advanced financial health indicators
  """

  # Credit health score
  df['Credit_Health_Score'] = (
      (df['Credit_Limit']/ df['Credit_Limit'].max()) * 0.3 +
      (1 - df['Avg_Utilization_Ratio']) * 0.4 +
      (df['Total_Trans_Ct']/df['Total_Trans_Ct'].max()) * 0.3
  )

  # Payment capacity ratio
  df['Payment_Capacity'] = df['Avg_Open_To_Buy'] / df['Credit_Limit']

  # Transaction efficiency
  df['Transaction_Efficiency'] = df['Total_Trans_Amt'] / df['Total_Trans_Ct']

  # Relationship tenure value
  df['Tenure_Value_Ratio'] = df['Total_Trans_Ct'] / df['Months_on_book']

  return df

## Behavioral Pattern Features

In [ ]:
def create_behavioural_features(df):
  """
  Create behavioural patterns indicators

  """
  # Activity consistency
  df['Activity_Consistency'] = 1/(1+df['Months_Inactive_12_mon'])

  # Service interaction intensity
  df['Service_Intensity'] = df['Contacts_Count_12_mon'] / 12

  # Usage volatility (Q4 vs Q1 changes)
  df['Usage_Volatility'] =  abs(df['Total_Amt_Chng_Q4_Q1'] -1) + abs(df['Total_Ct_Chng_Q4_Q1'] - 1)

  # Cross-product engagement
  df['Cross_Product_Engagement'] = df['Total_Relationship_Count'] / 6 # Normalized

  return df

## Risk Scoring Features

In [ ]:
def create_risk_features(df):
  """
  Create risk scoring features

  """

  # High utilization risk
  df['High_Util_Risk'] = (df['Avg_Utilization_Ratio'] > 0.8).astype(int)

  # Declining usage risk
  df['Declining_Usage_Risk'] = ((df['Total_Amt_Chng_Q4_Q1'] < 0.7) | (df['Total_Ct_Chng_Q4_Q1'] < 0.7)).astype(int)

  # Single product risk
  df['Single_Product_Risk'] = (df['Total_Relationship_Count'] == 1).astype(int)

  return df

## Encoding Features

In [ ]:
def encoding(df):
  """
  Encode categorical variables

  """
  # Gender encoded into numerical values
  df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})

  # Education_Level encoded into numerical values
  df['Education_Level'] = df['Education_Level'].map({'High School': 0, 'Graduate': 1, 'Uneducated': 2, 'College': 3, 'Post-Graduate': 4, 'Doctorate': 5})

  # Marital_Status encoded into numerical values
  df['Marital_Status'] = df['Marital_Status'].map({'Married': 0, 'Single': 1, 'Divorced': 2})
  # Income_Category encoded into numerical values
  df['Income_Category'] = df['Income_Category'].map({'Less than $40K': 0, '$40K - $60K': 1, '$60K - $80K': 2, '$80K - $120K': 3, '$120K +': 4})
  # Card_Category encoded into numerical values
  df['Card_Category'] = df['Card_Category'].map({'Blue': 0, 'Silver': 1, 'Gold': 2, 'Platinum': 3})
  # Utilization_Group encoded into numerical values
  df['Utilization_Group'] = df['Utilization_Group'].map({'Low (0-10%)': 0, 'Moderate (10-30%)': 1, 'High (30-70%)': 2, 'Very High (70%+)': 3})
  # Attrition_Flag encoded into numerical values
  df['Attrition_Flag'] = df['Attrition_Flag'].map({'Existing Customer': 0, 'Attrited Customer': 1})

  return df

## Customer Segmentation (RFM Analysis)

For credit card, I modified traditional RFM:

*   R (Recency): Days since last transaction
*   F (Frequency): Transaction count in last 12 months
*   M (Monetary): Total transaction amount in last 12 months

In [ ]:
def calculate_rfm_scores(df, transaction_date_col='Last_Transaction_Date'):
    """
    Calculate RFM scores for credit card customers

    """
    # Calculate Recency (days since last transaction)
    current_date = pd.to_datetime('2025-01-31') # Reference date
    df['Recency'] = (current_date - pd.to_datetime(df[transaction_date_col])).dt.days

    # Frequency is already available as Total_Trans_Ct
    df['Frequency'] = df['Total_Trans_Ct']

    # Monetary is already available as Total_Trans_Amt
    df['Monetary'] = df['Total_Trans_Amt']

    # Create quintile scores (1-5, where 5 is best)
    df['R_Score'] = pd.qcut(df['Recency'], 5, labels=[5,4,3,2,1]) # Lower recency = higher score
    df['F_Score'] = pd.qcut(df['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
    df['M_Score'] = pd.qcut(df['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])

    # Convert to numeric
    df['R_Score'] = df['R_Score'].astype(int)
    df['F_Score'] = df['F_Score'].astype(int)
    df['M_Score'] = df['M_Score'].astype(int)

    # Create RFM segments
    df['RFM_Score'] = df['R_Score'].astype(str) + df['F_Score'].astype(str) + df['M_Score'].astype(str)

    return df

In [ ]:
def create_cc_segments(df):
    """
    Create credit card specific customer segments

    """
    # Define segment rules based on RFM scores
    def assign_segment(row):

      r, f, m = row['R_Score'], row['F_Score'], row['M_Score']

      # Champions: High value, frequent, recent users
      if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
      # Loyal Customers: Regular users with good value
      elif r >= 3 and f >= 3 and m >= 3:
        return 'Loyal Customers'
      # Potential Loyalists: Good recent activity, building frequency
      elif r >= 4 and f >= 3 and m >= 2:
        return 'Potential Loyalists'
      # At Risk: Previously good customers, declining activity
      elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
      # Cannot Lose Them: High value but very low recent activity
      elif r <= 2 and f >= 4 and m >= 4:
        return 'Cannot Lose Them'
      # New Customers: Recent but low frequency/value
      elif r >= 4 and f <= 2 and m <= 2:
        return 'New Customers'
      # Hibernating: Low scores across all dimensions
      elif r <= 2 and f <= 2 and m <= 2:
        return 'Hibernating'
      # Need Attention: Moderate scores but concerning patterns
      else:
        return 'Need Attention'

    df['Customer_Segment'] = df.apply(assign_segment, axis=1)

    # Create dummy variables for each segment
    df['Customer_Segment'] = df['Customer_Segment'].map({'Champions': 1, 'Loyal Customers': 2, 'Potential Loyalists': 3, 'At Risk': 4, 'Cannot Lose Them': 5, 'New Customers': 6, 'Hibernating': 7, 'Need Attention': 8})


    return df

In [ ]:
def time_based_features(df):

    df['Spending_Trend'] = df['Total_Amt_Chng_Q4_Q1']
    df['Activity_Trend'] = df['Total_Ct_Chng_Q4_Q1']
    df['Declining_Spend_Flag'] = (df['Total_Amt_Chng_Q4_Q1'] < 1).astype(int)
    df['Declining_Activity_Flag'] = (df['Total_Ct_Chng_Q4_Q1'] < 1).astype(int)

    return df

In [ ]:
def target_repostion(df):
    """
    Reposition the target variable to the last column

    """

    attrition_flag = df.pop('Attrition_Flag')
    df['Attrition_Flag'] = attrition_flag

    return df

In [ ]:
if __name__ == '__main__':
  cleaned_data = create_financial_features(cleaned_data)
  cleaned_data = create_behavioural_features(cleaned_data)
  cleaned_data = create_risk_features(cleaned_data)
  cleaned_data = encoding(cleaned_data)
  cleaned_data = calculate_rfm_scores(cleaned_data)
  cleaned_data = create_cc_segments(cleaned_data)
  cleaned_data = time_based_features(cleaned_data)
  cleaned_data = target_repostion(cleaned_data)
  cleaned_data.to_csv('/content/drive/MyDrive/Customer_Churn/cleaned/final_data.csv', index=False)
  print("✅ Succesfully created the final dataset")

✅ Succesfully created the final dataset


In [ ]:
cleaned_data.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Last_Transaction_Date,Utilization_Group,Credit_Health_Score,Payment_Capacity,Transaction_Efficiency,Tenure_Value_Ratio,Activity_Consistency,Service_Intensity,Usage_Volatility,Cross_Product_Engagement,High_Util_Risk,Declining_Usage_Risk,Single_Product_Risk,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Customer_Segment,Spending_Trend,Activity_Trend,Declining_Spend_Flag,Declining_Activity_Flag,Attrition_Flag
0,49,0,2,0.0,0.0,0.0,1,34,4,2,0,4710.0,3137.0,1573.0,0.550,2448.0,38,0.817,0.666,2023-11-05,2,0.249587,0.333970,64.421053,1.117647,0.333333,0.000000,0.633,0.666667,0,1,0,453,38,2448.0,4,2,2,422,6,0.550,0.817,1,1,0
1,30,1,2,0.0,0.0,1.0,0,30,3,1,0,6038.0,824.0,5214.0,0.520,10069.0,26,1.099,0.136,2022-08-14,1,0.437027,0.863531,387.269231,0.866667,0.500000,0.000000,0.579,0.500000,0,1,0,901,26,10069.0,1,1,5,115,8,0.520,1.099,1,0,1
2,41,1,1,0.0,2.0,1.0,0,50,2,1,1,3924.0,3044.0,880.0,1.113,6389.0,112,0.674,0.776,2023-02-12,3,0.388772,0.224261,57.044643,2.240000,0.500000,0.083333,0.439,0.333333,0,1,0,719,112,6389.0,2,5,4,254,4,1.113,0.674,0,1,0
3,43,1,3,3.0,0.0,2.0,0,40,4,3,0,9226.0,855.0,8371.0,0.157,14261.0,106,1.160,0.093,2023-01-05,0,0.669573,0.907327,134.537736,2.650000,0.250000,0.000000,1.003,0.666667,0,1,0,757,106,14261.0,2,5,5,255,4,0.157,1.160,1,0,1
4,37,0,2,4.0,1.0,0.0,0,26,4,1,2,2734.0,1143.0,1591.0,0.736,3046.0,30,0.755,0.418,2024-01-15,2,0.320149,0.581931,101.533333,1.153846,0.500000,0.166667,0.509,0.666667,0,0,0,382,30,3046.0,4,2,2,422,6,0.736,0.755,1,1,0
